In [1]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [2]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [3]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCNO', 'des_col': 'scno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_SCNO', 'des_col': 'old_scno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NAME', 'des_col': 'name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FATHER_NAME', 'des_col': 'father_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GFATHER_NAME', 'des_col': 'gfather_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_1', 'des_col': 'address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_2', 'des_col': 'address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_3', 'des_col': 'address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_4', 'des_col': 'address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHONE', 'des_col': 'phone', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EMAIL', 'des_col': 'email', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'HOUSE_OWNER_NAME', 'des_col': 'house_owner_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'HOUSE_OWNER_FNAME', 'des_col': 'house_owner_fname', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AREA_CODE', 'des_col': 'area_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ROUTE', 'des_col': 'route', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CATEGORY', 'des_col': 'category', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUB_CATEGORY', 'des_col': 'sub_category', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHASE', 'des_col': 'phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONTRACTED_LOAD', 'des_col': 'contracted_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CONNECTED_LOAD', 'des_col': 'connected_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LOAD_UNIT', 'des_col': 'load_unit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOAD_TYPE', 'des_col': 'load_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NATURE_OF_PREMISES', 'des_col': 'nature_of_premises', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUPP_RELEASE_DATE', 'des_col': 'supp_release_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'SERVICE_STATUS', 'des_col': 'service_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSUMER_STATUS', 'des_col': 'consumer_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSUMER_TYPE', 'des_col': 'consumer_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POL_NUM', 'des_col': 'pol_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANS_LOCATION', 'des_col': 'trans_location', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FEEDER_NUM', 'des_col': 'feeder_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBSTATION_NUM', 'des_col': 'substation_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_1', 'des_col': 'alternate_address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_2', 'des_col': 'alternate_address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_3', 'des_col': 'alternate_address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_4', 'des_col': 'alternate_address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILLING_CYCLE', 'des_col': 'billing_cycle', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'SHORT_NAME', 'des_col': 'short_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AGREEMENT_DATE', 'des_col': 'agreement_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'LIGHTING_LOAD', 'des_col': 'lighting_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'TOWNSHIP_LOAD', 'des_col': 'township_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'TR_HIRE_CHARGES', 'des_col': 'tr_hire_charges', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'NO_OF_FEEDERS', 'des_col': 'no_of_feeders', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'ADDL_TEMP_LOAD', 'des_col': 'addl_temp_load', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUPPLY_VOLTAGE', 'des_col': 'supply_voltage', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'VOLTAGE_UNIT', 'des_col': 'voltage_unit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSTRUCTION_TYPE', 'des_col': 'construction_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CAPACITOR_AVAIL', 'des_col': 'capacitor_avail', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_OWNERSHIP', 'des_col': 'meter_ownership', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SBRNCH_CD', 'des_col': 'sbrnch_cd', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREDIT_HISTORY', 'des_col': 'credit_history', 'src_dtype': 'character', 'des_dtype': 'character'},
    {'src_col': 'CHEQUEBOUNCE_DATE', 'des_col': 'chequebounce_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CHEQUEBOUNCE_REASON', 'des_col': 'chequebounce_reason', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DEBT_FLAG', 'des_col': 'debt_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DEBT_ACTION_FLAG', 'des_col': 'debt_action_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALLOWEDCHKCNT', 'des_col': 'allowedchkcnt', 'src_dtype': 'DECIMAL', 'des_dtype': 'bigint'},
    {'src_col': 'ACCOUNT_NO', 'des_col': 'account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSUMER_ID', 'des_col': 'consumer_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COUNTRY_ID', 'des_col': 'country_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METERINGPOINT', 'des_col': 'meteringpoint', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONNECTIONEXPIREDATE', 'des_col': 'connectionexpiredate', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'APPROVEDPOWERFACTOR', 'des_col': 'approvedpowerfactor', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SERVICEREGION', 'des_col': 'serviceregion', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ISCONSUMEROPTEDTOD', 'des_col': 'isconsumeroptedtod', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEXTBILLINGDATE', 'des_col': 'nextbillingdate', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'BILLING_STATUS', 'des_col': 'billing_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCHEDULE_DAY', 'des_col': 'schedule_day', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'GEN_SCNO', 'des_col': 'gen_scno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_5', 'des_col': 'address_5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_5', 'des_col': 'alternate_address_5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EFFECT_DATE', 'des_col': 'effect_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'ROUTE_ID', 'des_col': 'route_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MR_APPLICABLE_FLAG', 'des_col': 'mr_applicable_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_RENT', 'des_col': 'meter_rent', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SUBMETER_COUNT', 'des_col': 'submeter_count', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'CONN_BY', 'des_col': 'conn_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LEVEL_ID', 'des_col': 'level_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILLFLG', 'des_col': 'billflg', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'DEMAND_METER_FLAG', 'des_col': 'demand_meter_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DEMAND', 'des_col': 'demand', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MTRNO', 'des_col': 'mtrno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MF_KWH', 'des_col': 'mf_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'GOVTCODE_ID', 'des_col': 'govtcode_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NOOFMONTHS_METERRENT', 'des_col': 'noofmonths_meterrent', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'BILLMONTH', 'des_col': 'billmonth', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'BILLYEAR', 'des_col': 'billyear', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'ETISALAT_ID', 'des_col': 'etisalat_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHONE_2', 'des_col': 'phone_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LANDMARKS', 'des_col': 'landmarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_GOVT_SUBSIDY', 'des_col': 'is_govt_subsidy', 'src_dtype': 'character', 'des_dtype': 'String'},
    {'src_col': 'GOVT_SUBSIDY', 'des_col': 'govt_subsidy', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DC_APP_FLG', 'des_col': 'dc_app_flg', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GISUID', 'des_col': 'gisuid', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DIESEL_CHG', 'des_col': 'diesel_chg', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PIN_NO', 'des_col': 'pin_no', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PRIORITY', 'des_col': 'priority', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BOOKNO', 'des_col': 'bookno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GIS_LAT', 'des_col': 'gis_lat', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'GIS_LONG', 'des_col': 'gis_long', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CHQ_REJ_MONTHS', 'des_col': 'chq_rej_months', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SET1', 'des_col': 'set1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MOHALLANO', 'des_col': 'mohallano', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GEOCODE', 'des_col': 'geocode', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METERING_VOLT', 'des_col': 'metering_volt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'FLORICARE_FLAG', 'des_col': 'floricare_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POWER_LOOM_FLAG', 'des_col': 'power_loom_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MET_DOM_LMV1_FLAG', 'des_col': 'met_dom_lmv1_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RURAL_FEEDER_FLAG', 'des_col': 'rural_feeder_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GOVT_CODE_ID', 'des_col': 'govt_code_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILLBASIS', 'des_col': 'billbasis', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PERMANENT_ADDRESS_1', 'des_col': 'permanent_address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PERMANENT_ADDRESS_2', 'des_col': 'permanent_address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PERMANENT_ADDRESS_3', 'des_col': 'permanent_address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METRIC', 'des_col': 'metric', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ABADI', 'des_col': 'abadi', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'VOLT_ID', 'des_col': 'volt_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPOUND_STATUS', 'des_col': 'compound_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PERMANENT_ADDRESS_4', 'des_col': 'permanent_address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PERMANENT_ADDRESS_5', 'des_col': 'permanent_address_5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INITIAL_ARRAMT', 'des_col': 'initial_arramt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'BINDER_NO', 'des_col': 'binder_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BINDER_PAGE_NO', 'des_col': 'binder_page_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOCATION_TYPE', 'des_col': 'location_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TYPE_SWITCH', 'des_col': 'type_switch', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_ROUNDCLOCK_SUP', 'des_col': 'is_roundclock_sup', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_SWH_EXISTS', 'des_col': 'is_swh_exists', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISTRICT', 'des_col': 'district', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TOWN', 'des_col': 'town', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'VILLAGE', 'des_col': 'village', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BA_DISTRICT', 'des_col': 'ba_district', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BA_TOWN', 'des_col': 'ba_town', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BA_VILLAGE', 'des_col': 'ba_village', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PA_DISTRICT', 'des_col': 'pa_district', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PA_TOWN', 'des_col': 'pa_town', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PA_VILLAGE', 'des_col': 'pa_village', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBGROUP', 'des_col': 'subgroup', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GRP', 'des_col': 'grp', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_REQUIRED', 'des_col': 'meter_required', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RETURN_MONTH', 'des_col': 'return_month', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DLIST_STATUS', 'des_col': 'dlist_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_RDG_CHNG', 'des_col': 'mtr_rdg_chng', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEXT_BILLINGDT', 'des_col': 'next_billingdt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'ISBILLED', 'des_col': 'isbilled', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DEDICATED_FLAG', 'des_col': 'dedicated_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANSFORMER_CAPACITY', 'des_col': 'transformer_capacity', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ACCOUNT_NO', 'des_col': 'new_account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AVG_UNITS', 'des_col': 'avg_units', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DEFAVG_UNITS', 'des_col': 'defavg_units', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'ZONE_ID', 'des_col': 'zone_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANSFERED_TO_ACNO', 'des_col': 'transfered_to_acno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANSFERED_BY', 'des_col': 'transfered_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANSFERED_DATE', 'des_col': 'transfered_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'TRANSFERED_REASON', 'des_col': 'transfered_reason', 'src_dtype': 'String', 'des_dtype': 'String'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.CMTCONSUMER_M', 'public', 'cmtconsumer_m', cols_map)
print("Total Migrated Records (cmtconsumer_m): " + str(migrated_row_count))


Total Migrated Records (cmtconsumer_m): Total Src Records: 75485 Total migrated Records to dest: 75485
